<a href="https://colab.research.google.com/github/metasebiya/Birhan-Engergies-week10/blob/task-2/src/baysian_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymc3==3.11.5 arviz==0.16.0 --quiet
!pip install pandas matplotlib==3.5.3 seaborn==0.11.2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 66.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a versio

In [ ]:
import os
os.environ = 'GNU' # Keep this line
os.environ = 'blas.check_openmp=False' # Add this line
# Then proceed with your imports and code
import pandas as pd
import numpy as np
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- 1. Data Preparation and EDA ---

# Load the Brent oil prices data
# Assuming 'BrentOilPrices.csv' is available in the execution environment
try:
    df = pd.read_csv('drive/Tenx Works/week 10/BrentOilPrices.csv')
except FileNotFoundError:
    print("Error: BrentOilPrices.csv not found. Please ensure the file is in the correct directory.")
    # Create a dummy DataFrame for demonstration if file is not found
    data = {
        'Date': pd.to_datetime(['1990-01-01', '1990-01-02', '1990-01-03', '2008-01-01', '2008-01-02', '2008-01-03', '2020-01-01', '2020-01-02', '2020-01-03']),
        'Price': [20.0, 20.1, 20.2, 100.0, 99.5, 99.0, 50.0, 49.5, 49.0]
    }
    df = pd.DataFrame(data)
    print("Using dummy data for demonstration.")

# Convert 'Date' column to datetime objects
df = pd.to_datetime(df, format='%d-%b-%y', errors='coerce')

# # Drop rows with NaT (Not a Time) if any date conversion failed
# df.dropna(subset=, inplace=True)

# Sort data by date
df = df.sort_values(by='Date').reset_index(drop=True)

# Calculate log returns for change point detection, especially for volatility changes
# Log returns are often more stationary and better for modeling volatility
df = np.log(df['Price'] / df['Price'].shift(1))
# # Drop the first row which will have NaN for Log_Return
# df.dropna(subset=, inplace=True)

# Use the log returns for the model
data_to_model = df.values
n_data_points = len(data_to_model)


In [ ]:
# Visual inspection of raw prices and log returns
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(df, df['Price'], label='Brent Oil Price')
plt.title('Brent Oil Prices Over Time')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True)
plt.legend()

In [ ]:
plt.subplot(2, 1, 2)
plt.plot(df, data_to_model, label='Daily Log Returns', color='orange')
plt.title('Daily Log Returns of Brent Oil Prices')
plt.xlabel('Date')
plt.ylabel('Log Return')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# --- 2. Implementing the Bayesian Change Point Model (PyMC3) ---

# Define the Bayesian model
with pm.Model() as change_point_model:
    # The unknown switch point (tau)
    # Uniform prior over all possible data points (excluding the very first and last few for stability)
    tau = pm.DiscreteUniform('tau', lower=0, upper=n_data_points - 1)

    # Parameters for the period before the change point
    # Mean of log returns before tau
    mu_before = pm.Normal('mu_before', mu=0, sd=0.1)
    # Standard deviation of log returns before tau (must be positive)
    sigma_before = pm.HalfNormal('sigma_before', sd=0.1)

    # Parameters for the period after the change point
    # Mean of log returns after tau
    mu_after = pm.Normal('mu_after', mu=0, sd=0.1)
    # Standard deviation of log returns after tau (must be positive)
    sigma_after = pm.HalfNormal('sigma_after', sd=0.1)

    # Combine parameters using pm.math.switch based on tau
    # This creates a piecewise function for mean and standard deviation
    idx = np.arange(n_data_points) # Array of indices for data points
    mu = pm.math.switch(idx < tau, mu_before, mu_after)
    sigma = pm.math.switch(idx < tau, sigma_before, sigma_after)

    # Likelihood function: observed log returns are normally distributed
    # with mean and standard deviation determined by the switch function
    observation = pm.Normal('observation', mu=mu, sd=sigma, observed=data_to_model)

    # Run the MCMC sampler
    # tune: number of tuning steps (discarded)
    # draws: number of samples to draw from the posterior
    # chains: number of independent chains to run
    # target_accept: target acceptance ratio for NUTS sampler
    trace = pm.sample(draws=2000, tune=1000, chains=2, target_accept=0.9, random_seed=42)



In [ ]:
# --- 3. Interpreting the Model Output ---

# Check for convergence and summarize results
print("\n--- Model Summary ---")
print(az.summary(trace, var_names=['tau', 'mu_before', 'sigma_before', 'mu_after', 'sigma_after']))

# Plot trace for visual convergence check
print("\n--- Trace Plots (for convergence diagnostics) ---")
az.plot_trace(trace, var_names=['tau', 'mu_before', 'sigma_before', 'mu_after', 'sigma_after'])
plt.tight_layout()
plt.show()

# Plot posterior distribution of tau to identify the most probable change point
print("\n--- Posterior Distribution of Change Point (tau) ---")
az.plot_posterior(trace, var_names=['tau'], kind='hist', bins=50)
plt.title('Posterior Distribution of Change Point (tau)')
plt.xlabel('Index of Change Point')
plt.ylabel('Probability Density')
plt.show()

# Get the most probable change point index
tau_posterior_mean = int(trace['tau'].mean())
tau_posterior_mode = int(trace['tau'].mode()) # Mode is often more indicative for discrete variables

print(f"\nMost probable change point index (mean): {tau_posterior_mean}")
print(f"Most probable change point index (mode): {tau_posterior_mode}")

# Convert the index to a date
# Adjusting for the log_return shift (first row was dropped)
if tau_posterior_mode + 1 < len(df):
    change_point_date = df.iloc[tau_posterior_mode + 1]
    print(f"Most probable change point date: {change_point_date.strftime('%Y-%m-%d')}")
else:
    print("Change point index out of bounds for date conversion.")

# Plot posterior distributions of parameters to quantify impact
print("\n--- Posterior Distributions of Mean and Standard Deviation Parameters ---")
az.plot_posterior(trace, var_names=['mu_before', 'mu_after'], kind='hist', bins=30)
plt.title('Posterior Distributions of Mean Log Returns')
plt.show()

az.plot_posterior(trace, var_names=['sigma_before', 'sigma_after'], kind='hist', bins=30)
plt.title('Posterior Distributions of Standard Deviation of Log Returns')
plt.show()

# Quantify the impact (e.g., mean and standard deviation values)
mean_before = trace['mu_before'].mean()
mean_after = trace['mu_after'].mean()
sigma_before = trace['sigma_before'].mean()
sigma_after = trace['sigma_after'].mean()

print(f"\nMean log return before change point: {mean_before:.4f}")
print(f"Mean log return after change point: {mean_after:.4f}")
print(f"Standard deviation of log return before change point: {sigma_before:.4f}")
print(f"Standard deviation of log return after change point: {sigma_after:.4f}")

# Calculate percentage change in standard deviation (as a proxy for volatility change)
percent_change_sigma = ((sigma_after - sigma_before) / sigma_before) * 100
print(f"Percentage change in volatility (standard deviation): {percent_change_sigma:.2f}%")

In [ ]:
# --- 4. Visualizing the Change Point on the Price Series ---
plt.figure(figsize=(14, 6))
plt.plot(df, df['Price'], label='Brent Oil Price')
if 'change_point_date' in locals():
    plt.axvline(x=change_point_date, color='r', linestyle='--', label=f'Detected Change Point: {change_point_date.strftime("%Y-%m-%d")}')
plt.title('Brent Oil Prices with Detected Change Point')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True)
plt.legend()
plt.show()